In [ ]:
import pandas as pd
import numpy as np
import os
import statistics
import openpyxl
from scipy.stats import randint
import sys
import statsmodels.api as sm
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, RandomizedSearchCV, KFold, GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from scipy.stats import randint
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.preprocessing import MaxAbsScaler, OneHotEncoder, StandardScaler
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import mean_squared_error

In [ ]:
!pip install -U scikit-learn
!pip install pickle5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Abertura dos dados

In [ ]:
treino = pd.read_excel('/content/drive/MyDrive/Mestrado/Resultados.xlsx', sheet_name = 'Dados_Treino')
teste = pd.read_excel('/content/drive/MyDrive/Mestrado/Resultados.xlsx', sheet_name = 'Dados_Teste')

In [ ]:
dados_treino = treino[['Amostra',
                                           'Litofacies',
                                           'Categoria Litofacies',
                                           'Porosidade Gas',
                                           'Porosidade RMN',
                                           'Permeabilidade Gas',
                                           'T2 Ponderado Log',
                                           'S1',
                                           'S2',
                                           'S3',
                                           'S4']].replace(0, 0.00001)

dados_teste = teste[['Amostra',
                                           'Litofacies',
                                           'Categoria Litofacies',
                                           'Porosidade Gas',
                                           'Porosidade RMN',
                                           'Permeabilidade Gas',
                                           'T2 Ponderado Log',
                                           'S1',
                                           'S2',
                                           'S3',
                                           'S4']].replace(0, 0.00001)

In [ ]:
def ProcessamentoDadosSDR (Dataframe):
    dados = pd.DataFrame({
        'Amostra': Dataframe['Amostra'],
        'Litofacies': Dataframe['Litofacies'],
        'T2': Dataframe['T2 Ponderado Log'],
        'Porosidade RMN': Dataframe['Porosidade RMN'],
        'Porosidade Gas': Dataframe['Porosidade Gas'],
        'Permeabilidade Gas': Dataframe['Permeabilidade Gas']
    })
    return dados

def RegressaoSDR (Dataframe_SDR):
    # Regressão via OLS
    t2 = Dataframe_SDR['T2 Ponderado Log']
    phi = Dataframe_SDR['Porosidade RMN']
    permeabilidade = Dataframe_SDR['Permeabilidade Gas']
    dados_calculo = pd.DataFrame({'Log k': np.log(permeabilidade),
                                'Log φ': np.log(phi),
                                'Log T2': np.log(t2)})
    dados_calculo = sm.add_constant(dados_calculo)
    atributos = dados_calculo[['const', 'Log φ', 'Log T2']]
    rotulos = dados_calculo[['Log k']]
    reg_ols_log = sm.OLS(rotulos, atributos, hasconst=True).fit()

    # Obtenção dos coeficientes da Regressão
    coeficientes = pd.DataFrame({
        'Coeficiente': ['a', 'b', 'c', 'R2'],
        'Valor': [np.exp(reg_ols_log.params[0]),
                  reg_ols_log.params[1],
                  reg_ols_log.params[2],
                  reg_ols_log.rsquared]}).set_index('Coeficiente')

    # Cálculo da Previsão com base nos coeficientes obtidos
    a = coeficientes['Valor']['a']
    b = coeficientes['Valor']['b']
    c = coeficientes['Valor']['c']
    k = (a*(phi**b)*(t2**c))
    dados = pd.DataFrame({'Permeabilidade Prevista': k})

    #Erro Sigma
    k_p = np.log10(dados['Permeabilidade Prevista'])
    k_g = np.log10(permeabilidade)
    N = len(k_p)
    soma = np.sum((k_p-k_g)**2)
    raiz = np.sqrt(soma/N)
    sigma = 10**(raiz)

    return reg_ols_log, coeficientes, pd.concat([Dataframe_SDR, dados], axis = 1), sigma

def ProcessamentoDadosHan (Dataframe):
    dados = pd.DataFrame({'Amostra': Dataframe['Amostra'],
                          'Litofacies': Dataframe['Litofacies'],
                          'Permeabilidade Gas': Dataframe['Permeabilidade Gas'],
                          'Porosidade Gas': Dataframe['Porosidade Gas'],
                          'Porosidade RMN': Dataframe['Porosidade RMN'],
                          'S1': Dataframe['S1'],
                          'S2': Dataframe['S2'],
                          'S3': Dataframe['S3'],
                          'S4': Dataframe['S4']}).replace(0, np.nan).dropna().reset_index().drop('index', axis = 1)

    return dados

def RegressaoHan (Dataframe_Novo_SDR):
    # Regressão via OLS
    dados_calculo_log = pd.DataFrame({
    'Log k': np.log(Dataframe_Novo_SDR['Permeabilidade Gas']),
    'Log φ': np.log(Dataframe_Novo_SDR['Porosidade RMN']),
    'S1 log': (-1)*(np.log(Dataframe_Novo_SDR['S1'])),
    'S2 log': (-1)*(np.log(Dataframe_Novo_SDR['S2'])),
    'S3 log': np.log(Dataframe_Novo_SDR['S3']),
    'S4 log': np.log(Dataframe_Novo_SDR['S4'])})
    dados_calculo = sm.add_constant(dados_calculo_log)

    atributos = dados_calculo[['const', 'Log φ', 'S3 log', 'S4 log', 'S1 log', 'S2 log']]
    rotulos = dados_calculo['Log k']
    reg_novo = sm.OLS(rotulos, atributos, hasconst=True, missing = 'drop').fit()

    # Obtenção dos coeficientes da Regressão
    coeficientes_novo = pd.DataFrame({
          'Coeficiente': ['a', 'b', 'c', 'd', 'e', 'f', 'R2'],
          'Valor': [np.exp(reg_novo.params[0]),
                    reg_novo.params[1],
                    reg_novo.params[2],
                    reg_novo.params[3],
                    reg_novo.params[4],
                    reg_novo.params[5],
                    reg_novo.rsquared]
          }).set_index('Coeficiente')

    # Cálculo da Previsão com base nos coeficientes obtidos
    a = coeficientes_novo['Valor']['a']
    b = coeficientes_novo['Valor']['b']
    c = coeficientes_novo['Valor']['c']
    d = coeficientes_novo['Valor']['d']
    e = coeficientes_novo['Valor']['e']
    f = coeficientes_novo['Valor']['f']
    phi = Dataframe_Novo_SDR['Porosidade RMN']
    s1 = Dataframe_Novo_SDR['S1']
    s2 = Dataframe_Novo_SDR['S2']
    s3 = Dataframe_Novo_SDR['S3']
    s4 = Dataframe_Novo_SDR['S4']
    k = a*(phi**b)*(s3**c)*(s4**d)/((s1**e)*(s2**f))
    dados = pd.DataFrame({'Permeabilidade Prevista': k})

    #Erro Sigma
    k_p = np.log10(dados['Permeabilidade Prevista'])
    k_g = np.log10(Dataframe_Novo_SDR['Permeabilidade Gas'])
    N = len(k_p)
    soma = np.sum((k_p-k_g)**2)
    raiz = np.sqrt(soma/N)
    sigma = 10**(raiz)



    return reg_novo, coeficientes_novo, pd.concat([Dataframe_Novo_SDR, dados], axis = 1), sigma

In [ ]:
treino['S4']

0      0.023175
1      0.004999
2      0.015698
3      0.007072
4      0.000933
         ...   
339    0.000959
340    0.002621
341    0.006431
342    0.005753
343    0.003873
Name: S4, Length: 344, dtype: float64

In [ ]:
reg_SDR, coef_SDR, df_SDR, erro_SDR = RegressaoSDR(treino)
reg_SDR_teste, coef_SDR_teste, df_SDR_teste, erro_SDR_teste = RegressaoSDR(teste)

In [ ]:
t2_teste = df_SDR_teste['T2 Ponderado Log']
phi_teste = df_SDR_teste['Porosidade RMN']

a = coef_SDR['Valor']['a']
b = coef_SDR['Valor']['b']
c = coef_SDR['Valor']['c']
k = (a*(phi_teste**b)*(t2_teste**c))

#Erro Sigma
k_p = np.log10(k)
k_g = np.log10(df_SDR_teste['Permeabilidade Gas'])
N = len(k_p)
soma = np.sum((k_p-k_g)**2)
raiz = np.sqrt(soma/N)
sigma_sdr = 10**(raiz)

sigma_sdr
df_SDR.to_excel('/content/drive/MyDrive/Mestrado/Resultados_SDR_Treino.xlsx')
df_SDR_teste.to_excel('/content/drive/MyDrive/Mestrado/Resultados_SDR_Teste.xlsx')

In [ ]:
print(a, b, c)

7.208683683661914 2.7098425363081513 0.4181978047603788


In [ ]:
reg_Han, coef_Han, df_Han, erro_Han = RegressaoHan(dados_treino)
reg_Han_teste, coef_Han_teste, df_Han_teste, erro_Han_teste = RegressaoHan(dados_teste)

In [ ]:
a = coef_Han['Valor']['a']
b = coef_Han['Valor']['b']
c = coef_Han['Valor']['c']
d = coef_Han['Valor']['d']
e = coef_Han['Valor']['e']
f = coef_Han['Valor']['f']
phi = df_Han_teste['Porosidade RMN']
s1 = df_Han_teste['S1']
s2 = df_Han_teste['S2']
s3 = df_Han_teste['S3']
s4 = df_Han_teste['S4']
k = a*(phi**b)*(s3**c)*(s4**d)/((s1**e)*(s2**f))
df_Han_teste['Permeabilidade Prevista'] = k

#Erro Sigma
k_p = np.log10(k)
k_g = np.log10(df_Han_teste['Permeabilidade Gas'])
N = len(k_p)
soma = np.sum((k_p-k_g)**2)
raiz = np.sqrt(soma/N)
sigma_han = 10**(raiz)
sigma_han

df_Han.to_excel('/content/drive/MyDrive/Mestrado/Resultados_Han_Treino.xlsx')
df_Han_teste.to_excel('/content/drive/MyDrive/Mestrado/Resultados_Han_Teste.xlsx')

In [ ]:
sigma_han

4.412010250906544

In [ ]:
print(a, b, c, d, e, f)

13.597514789200106 2.4287644333230727 0.0488340566000693 0.06839561796903659 1.0537568426651407 -0.0193906006585597


In [ ]:
treino = pd.read_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Resultados.xlsx', sheet_name = 'Dados_Treino')
teste = pd.read_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Resultados.xlsx', sheet_name = 'Dados_Teste')

In [ ]:
def VisualizarPredicoes (Dados, Pasta_Salvamento = None, Data= None, Modelo = None,
                          Litofacies = None, Salvar = False, Sigma = False, Valor_Sigma = 3.64):
    titulo = 'Predições da Permeabilidade ' + str(Modelo)
    eixo_x = 'Log Permeabilidade Gás (mD)'
    eixo_y = 'Log Permeabilidade RMN (mD)'
    reta = pd.DataFrame({'x' : np.arange(1000),
                         'y' : np.arange(1000)})
    plt.subplots(figsize = (9,6))
    sns.scatterplot(data = Dados,
                    x = 'Permeabilidade Gas',
                    y = 'Permeabilidade Prevista',
                    hue = Litofacies,
                    palette = 'Spectral')
    sns.lineplot(data = reta,
                 x = 'x',
                 y = 'y')
    if Sigma == True:
        plt.plot(reta['x'], reta['y'] * Valor_Sigma, "b-.", linewidth=1)
        plt.plot(reta['x'], reta['y'] / Valor_Sigma, "b-.", linewidth=1, label = f'+/- \u03C3: {Valor_Sigma:.2f}')

    plt.xlabel(eixo_x)
    plt.ylabel(eixo_y)
    plt.title(titulo)
    plt.xlim(0.00001,1000)
    plt.ylim(0.00001,1000)
    plt.legend(loc="upper left", fontsize=10)

    plt.xscale('log')
    plt.yscale('log')

    if Salvar == True:
        plt.savefig(Pasta_Salvamento + Data + titulo + '.png', format='png')

    plt.show()

In [ ]:
treino = pd.read_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Dados-Salvos/Resultados_treino.xlsx').drop(['Unnamed: 0.5',	'Unnamed: 0.4',	'Unnamed: 0.3',
                                                                                                              'Unnamed: 0.2',	'Unnamed: 0.1',	'Unnamed: 0'],
                                                                                                              axis = 1)
teste = pd.read_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Dados-Salvos/Resultados_teste.xlsx').drop(['Unnamed: 0.5',	'Unnamed: 0.4',	'Unnamed: 0.3',
                                                                                                              'Unnamed: 0.2',	'Unnamed: 0.1',	'Unnamed: 0'],
                                                                                                              axis = 1)

In [ ]:
df_Han.to_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Dados-Salvos/Resultado_Han.xlsx')
df_SDR.to_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Dados-Salvos/Resultado_SDR.xlsx')

In [ ]:
df_Han_teste['Amostra']

0      AFR3-B-14
1      AFR3-B-13
2      AFR1-D-03
3      AA01-A-13
4      AT04-A-04
         ...    
145    AA01-C-01
146    AA05-C-04
147    AT01-C-05
148    AT01-D-01
149    AT04-B-12
Name: Amostra, Length: 150, dtype: object

In [ ]:
treino['SDR'] = df_SDR['Permeabilidade Prevista']
treino['Han'] = df_Han['Permeabilidade Prevista']

teste['SDR'] = df_SDR_teste['Permeabilidade Prevista']
teste['Han'] = df_Han_teste['Permeabilidade Prevista']

In [ ]:
treino.to_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Dados-Salvos/Resultado_treino.xlsx')
teste.to_excel('/content/drive/MyDrive/Mestrado/Artigo - 1/Dados-Salvos/Resultado_teste.xlsx')

In [ ]:
VisualizarPredicoes(df_SDR, Sigma = True, Valor_Sigma = erro_SDR, Litofacies = 'Litofacies', Modelo = 'Treino SDR')

In [ ]:
VisualizarPredicoes(df_SDR_teste, Sigma = True, Valor_Sigma = sigma_sdr, Litofacies = 'Litofacies', Modelo = 'Teste SDR')

In [ ]:
VisualizarPredicoes(df_Han, Sigma = True, Valor_Sigma = erro_Han, Litofacies = 'Litofacies', Modelo = 'Treino Han')

In [ ]:
VisualizarPredicoes(df_Han_teste, Sigma = True, Valor_Sigma = sigma_han, Litofacies = 'Litofacies', Modelo = 'Teste Han')